In [1]:
%matplotlib inline
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [2]:
plt.rcParams["figure.figsize"] = (15, 10)

In [49]:
conn = sqlite3.connect("results/decentfl.1.db")

exp = pd.read_sql("select * from experiments;", conn)
eva = pd.read_sql("select exp.*, eva.* from experiments as exp, evaluations as eva where exp.id = eva.experiment_id", conn)

In [50]:
time_mins = eva.groupby(list(set(exp.columns) - set(["id", ""]))).min()[["timestamp"]].reset_index(inplace=False)
eva["timestamp"] = eva.transpose().apply(lambda x: x["timestamp"] - time_mins[(time_mins.group_size == x["group_size"]) & (time_mins.update_every == x["update_every"]) & (time_mins.seed == x["seed"])]["timestamp"].values[0])

In [52]:
eva_mean = eva.groupby(["update_every", "group_size", "step"]).mean()[["value", "timestamp"]]
eva_mean.reset_index(inplace=True)
eva_std = eva.groupby(["update_every", "group_size", "step"]).std()[["value", "timestamp"]]
eva_std.reset_index(inplace=True)

In [53]:
from ipywidgets import interact, IntSlider

# Single

In [55]:
@interact
def plot(
    update_every=IntSlider(min=eva_mean.update_every.min(), max=eva_mean.update_every.max()),
    group_size=IntSlider(min=eva_mean.group_size.min(), max=eva_mean.group_size.max()),
    timestamp=False
):
    data = eva_mean[(eva_mean.update_every == update_every) & (eva_mean.group_size == group_size)]
    data_std = eva_std[(eva_std.update_every == update_every) & (eva_std.group_size == group_size)]
    x = data.timestamp if timestamp else data.step
    plt.plot(x, data.value)
    plt.fill_between(x, data.value - data_std.value, data.value + data_std.value, alpha=0.3)
    plt.title(f"update every {update_every} epochs with group size {group_size}")
    plt.xlabel("Seconds" if timestamp else "Epoch")
    plt.ylabel("Test Accuracy")
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=1, description='update_every', max=5, min=1), IntSlider(value=3, descrip…

# Update Every

In [56]:
@interact
def plot(
    group_size=IntSlider(min=eva_mean.group_size.min(), max=eva_mean.group_size.max()),
    timestamp=False
):
    for update_every in eva_mean.update_every.unique():
        data = eva_mean[(eva_mean.update_every == update_every) & (eva_mean.group_size == group_size)]
        data_std = eva_std[(eva_std.update_every == update_every) & (eva_std.group_size == group_size)]
        x = data.timestamp if timestamp else data.step
        plt.plot(x, data.value, label=update_every)
        plt.fill_between(x, data.value - data_std.value, data.value + data_std.value, alpha=0.3)
    plt.title(f"update every nth epochs with group size {group_size}")
    plt.xlabel("Seconds" if timestamp else "Epoch")
    plt.ylabel("Test Accuracy")
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=3, description='group_size', max=9, min=3), Checkbox(value=False, descri…

# Group Size

In [57]:
@interact
def plot(
    update_every=IntSlider(min=eva_mean.update_every.min(), max=eva_mean.update_every.max()),
    timestamp=False
):
    for group_size in eva_mean.group_size.unique():
        data = eva_mean[(eva_mean.update_every == update_every) & (eva_mean.group_size == group_size)]
        data_std = eva_std[(eva_std.update_every == update_every) & (eva_std.group_size == group_size)]
        x = data.timestamp if timestamp else data.step
        plt.plot(x, data.value, label=group_size)
        plt.fill_between(x, data.value - data_std.value, data.value + data_std.value, alpha=0.3)
    plt.title(f"update every {update_every} epochs with group size n")
    plt.xlabel("Seconds" if timestamp else "Epoch")
    plt.ylabel("Test Accuracy")
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=1, description='update_every', max=5, min=1), Checkbox(value=False, desc…